In [232]:
import sys
from IPython.display import display
import numpy as np

# check if python3 is used
if sys.version_info < (3, 0):
    print("This programs need python3x to run. Aborting.")
    sys.exit(1)

import os
import glob
import random
from PIL import Image
from math import floor

def moments2e(image):
  """
  Most of this is from:
  https://github.com/shackenberg/Image-Moments-in-Python/blob/master/moments.py
  
  This function calculates the raw, centered and normalized moments
  for any image passed as a numpy array.
  Further reading:
  https://en.wikipedia.org/wiki/Image_moment
  https://en.wikipedia.org/wiki/Central_moment
  https://en.wikipedia.org/wiki/Moment_(mathematics)
  https://en.wikipedia.org/wiki/Standardized_moment
  http://opencv.willowgarage.com/documentation/cpp/structural_analysis_and_shape_descriptors.html#cv-moments
  
  compare with:
  import cv2
  cv2.moments(image)
  """
  from numpy import mgrid, sum
    
  # need it to be gray scale first
  gray = image.convert('L')  

  image = np.asarray(gray.point(lambda x: 0 if x<1 else 255, '1'), dtype='int8')
  assert len(image.shape) == 2 # only for grayscale images        
  x, y = mgrid[:image.shape[0],:image.shape[1]]
  moments = {}
  moments['mean_x'] = sum(x*image)/sum(image)
  moments['mean_y'] = sum(y*image)/sum(image)
          
  # raw or spatial moments
  moments['m00'] = sum(image)
  moments['m01'] = sum(x*image)
  moments['m10'] = sum(y*image)
  moments['m11'] = sum(y*x*image)
  moments['m02'] = sum(x**2*image)
  moments['m20'] = sum(y**2*image)
  moments['m12'] = sum(x*y**2*image)
  moments['m21'] = sum(x**2*y*image)
  moments['m03'] = sum(x**3*image)
  moments['m30'] = sum(y**3*image)
  
  # central moments
  # moments['mu01']= sum((y-moments['mean_y'])*image) # should be 0
  # moments['mu10']= sum((x-moments['mean_x'])*image) # should be 0
  moments['mu11'] = sum((x-moments['mean_x'])*(y-moments['mean_y'])*image)
  moments['mu02'] = sum((y-moments['mean_y'])**2*image) # variance
  moments['mu20'] = sum((x-moments['mean_x'])**2*image) # variance
  moments['mu12'] = sum((x-moments['mean_x'])*(y-moments['mean_y'])**2*image)
  moments['mu21'] = sum((x-moments['mean_x'])**2*(y-moments['mean_y'])*image) 
  moments['mu03'] = sum((y-moments['mean_y'])**3*image) 
  moments['mu30'] = sum((x-moments['mean_x'])**3*image) 

    
  # opencv versions
  #moments['mu02'] = sum(image*(x-moments['m01']/moments['m00'])**2)
  #moments['mu02'] = sum(image*(x-y)**2)

  # wiki variations
  #moments['mu02'] = m20 - mean_y*m10 
  #moments['mu20'] = m02 - mean_x*m01
        
  # central standardized or normalized or scale invariant moments
  moments['nu11'] = moments['mu11'] / sum(image)**(2/2+1)
  moments['nu12'] = moments['mu12'] / sum(image)**(3/2+1)
  moments['nu21'] = moments['mu21'] / sum(image)**(3/2+1)
  moments['nu20'] = moments['mu20'] / sum(image)**(2/2+1)
  moments['nu03'] = moments['mu03'] / sum(image)**(3/2+1) # skewness
  moments['nu30'] = moments['mu30'] / sum(image)**(3/2+1) # skewness
  #print(moments)
  return moments

def orientation(moments, out_type='deg'):
    """
    Returns orientation from moments, in degrees or radians
    """
    theta = 0.5*np.arctan(2*moments['mu11']/(moments['mu20']-moments['mu02']))
    
    if (out_type == 'deg'):
        return theta*(180/np.pi)
    else:
        return theta

def initialize(backgrounds_dir, classes_dir):
    # Loading data
    backgrounds = os.listdir(backgrounds_dir)
    class_names = os.listdir(classes_dir)
    class_objects = [os.listdir(os.path.join(classes_dir,c)) for c in class_names]

    # ignore scaling for now: .resize(resolution, Image.BICUBIC)
    bgs = [Image.open(os.path.join(backgrounds_dir,b)) for b in backgrounds]

    objs = []
    for i,c in enumerate(class_objects):
        print('loading '+class_names[i]+' as '+str(i))
        objs = objs + [[Image.open(os.path.join(classes_dir,class_names[i],o)) for o in c]]

    return objs, class_names, bgs



def scale_obj(obj, size_real=300, distance=200):
    # in front of image size is 1392px/339mm
    # in back of image size is 1392px/1190mm
         
    max_distance = 700
    
    pix_per_mm = (1392/(339 + (distance-200)*((1190-339)/(max_distance-200))))
    
    
    new_length = (int)(pix_per_mm*size_real)
    
    
    ratio = new_length/(int)(np.shape(obj)[1])
    print('distance ' +str(distance))
    
    print('pix per mm ', str(pix_per_mm))
    print('ratio ' + str(ratio))
    print('new length ' + str(new_length))    
    new_height = (int)(np.shape(obj)[0]*ratio)
    print('new height ' + str(new_height))
    
    #print(ratio, new_length, new_height)
    
    new_obj=obj.resize((new_length, new_height))
    
    return new_obj

# Simulate an image
def mkimage(filename, objs, names, bgs, maxobjs, output_dir="images_out",single=False):
    log = []
    im = bgs[random.randint(0,len(bgs)-1)].copy()
    # print('bg size='+str(im.size))
    cls0 = random.randint(0,len(objs)-1)
    for c in range(0,random.randint(1,maxobjs)):
        if single: cls=cls0
        else: cls = random.randint(0,len(objs)-1)
        obj = random.choice(objs[cls])        
        imx,imy = im.size
        
        # first paste to zero background, or the moment function
        # will return the wrong values
        black_bkg = Image.new('L', obj.size)
        black_bkg.paste(obj,(0,0),obj)
        
        
        
        # horizontally align objects so scaling will be appropriate
        degrees = orientation(moments2e(black_bkg))        
        obj_aligned = obj.rotate(-1*degrees,expand=True)
        
        # we probably padded it with a lot of black that will interfere with placement
        # so we need to get rid of this
        bbox = obj_aligned.getbbox()
        obj_aligned = obj_aligned.crop(bbox)
        
        
        sizex,sizey = obj_aligned.size
        print(obj_aligned.size)
        if(sizey > sizex):
            obj_aligned = obj_aligned.rotate(-90,expand=True)
        display(scale_obj(obj_aligned,size_real=200, distance=700))
        
        imx,imy = im.size
        posx = random.randint(-floor(sizex/2),imx-floor(sizex/2))
        posy = random.randint(-floor(sizey/2),imy-floor(sizey/2))
        im.paste(obj_aligned,(posx,posy),obj_aligned)
        log = log + ['{}\t{}\t{}\t{}\t{}\t{}\n'.format(names[cls],cls,posy,posx,posy+sizey,posx+sizex)]
    im.save(os.path.join(output_dir,filename+'.png'))
    with open(os.path.join(output_dir,filename+'.txt'),'w') as f:
        for l in log: f.write(l)

# Testing
def test():
    objects, names, backgrounds = initialize(backgrounds_dir="backgrounds", classes_dir="crops")
    mkimage('test', objects, names, backgrounds, output_dir="images_out", maxobjs=6)


In [231]:
objects, names, backgrounds = initialize(backgrounds_dir="source_data/Backgrounds", classes_dir="source_data/Classes")
mkimage('test', objects, names, backgrounds, output_dir="images_out", maxobjs=6)



loading Benthosema as 0
loading BlueWhiting as 1
loading Herring as 2
loading Mackerel as 3
(161, 34)
distance 700
pix per mm  1.169747899159664
ratio 0.7204968944099379
new length 116
new height 24


(894, 227)
distance 700
pix per mm  1.169747899159664
ratio 0.1297539149888143
new length 116
new height 29
